In [ ]:
import pathlib
import pandas as pd

# Ruta a los CSV 
DATA_DIR = pathlib.Path("../data/raw")

# Formato de impresión para floats (sólo estética)
pd.options.display.float_format = "{:,.2f}".format
pd.set_option('display.max_columns', None)


[INFO] Carpeta de datos: C:\Users\victo\Desktop\DATA_RETAIL\data\raw
[INFO] Buscando archivos CSV...


In [ ]:
print(f"[INFO] Carpeta de datos: {DATA_DIR.resolve()}")

In [3]:
csv_paths = list(DATA_DIR.glob("*.csv"))

print(f"[INFO] Encontrados {len(csv_paths)} CSV:")
for p in csv_paths:
    print("   -", p.name)

if not csv_paths:
    raise FileNotFoundError("No se encontraron CSV en ../data/raw. ¿Ruta correcta?")

[INFO] Encontrados 3 CSV:
   - Amazon Sale Report.csv
   - International sale Report.csv
   - Sale Report.csv


In [4]:
def cargar_todos_csv(path):
    """
    Intenta leer un CSV probando varios separadores y encodings.
    Devuelve (df, encoding, sep). Lanza ValueError si no puede leer.
    """

    archivo = []

    for enc in ("utf-8-sig", "utf-8", "latin-1", "utf-16"):

        for sep in (",", ";", "|", "\t"):

            try:
                df = pd.read_csv(path, encoding=enc, sep=sep)
                
                if df.shape[1] > 1: # al menos 2 columnas
                    print(f"[OK] {path.name}: encoding={enc}, sep='{sep}', shape={df.shape}")
                    return df, enc, sep
                else:
                    archivo.append((enc, sep, "solo 1 columna"))
            except Exception as e:
                archivo.append((enc, sep, str(e)))

    # En caso de error, mostrar intentos
    print(f"[ERROR] No pude leer {path.name}. Intentos:")

    for enc, sep, err in archivo[:10]:  # mostrar algunos intentos
        print(f"   enc={enc}, sep='{sep}' -> {err}")

    raise ValueError(f"No se pudo leer {path}")

In [ ]:
# Leer todos los CSV y guardar información 
df_list = []
info_list = []


for path in csv_paths:
    df, enc, sep = cargar_todos_csv(path)
    df["file_name"] = path.name # Creamos df con columna file_name

    df_list.append(df)
    info_list.append({
        "file": path.name,
        "rows": len(df),
        "cols": df.shape[1],
        "encoding": enc,
        "sep": sep,
    })


[OK] Amazon Sale Report.csv: encoding=utf-8-sig, sep=',', shape=(128975, 24)
[OK] International sale Report.csv: encoding=utf-8-sig, sep=',', shape=(37432, 10)
[OK] Sale Report.csv: encoding=utf-8-sig, sep=',', shape=(9271, 7)


C:\Users\victo\AppData\Local\Temp\ipykernel_32728\1527596446.py:14: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, encoding=enc, sep=sep)


In [6]:
# Mostrar un resumen de lo leído
print("\n[RESUMEN DE CARGA]")
for m in info_list:
    print(f" - {m['file']}: {m['rows']} filas x {m['cols']} cols (enc={m['encoding']}, sep='{m['sep']}')")


[RESUMEN DE CARGA]
 - Amazon Sale Report.csv: 128975 filas x 25 cols (enc=utf-8-sig, sep=',')
 - International sale Report.csv: 37432 filas x 11 cols (enc=utf-8-sig, sep=',')
 - Sale Report.csv: 9271 filas x 8 cols (enc=utf-8-sig, sep=',')


In [7]:
# Normalizar nombres de columnas 
def normalizar_nombres_columnas(name: str) -> str:
    name = name.strip()
    name = name.lower()
    name = name.replace(" ", "_")
    name = name.replace("%", "pct")
    name = name.replace("-", "_")
    return name

In [8]:
df_list_norm = []

for i, d in enumerate(df_list):

    old_cols = list(d.columns)
    d2 = d.rename(columns=lambda c: normalizar_nombres_columnas(str(c)))
    new_cols = list(d2.columns)

    if old_cols != new_cols:
        print(f"[INFO] Normalizadas columnas en DF {i}:")

        cambios = list(zip(old_cols, new_cols))

        for old, new in cambios:
            if old != new:
                print(f"   -   '{old}' -> '{new}'")


    df_list_norm.append(d2)

[INFO] Normalizadas columnas en DF 0:
   -   'Order ID' -> 'order_id'
   -   'Date' -> 'date'
   -   'Status' -> 'status'
   -   'Fulfilment' -> 'fulfilment'
   -   'Sales Channel ' -> 'sales_channel'
   -   'ship-service-level' -> 'ship_service_level'
   -   'Style' -> 'style'
   -   'SKU' -> 'sku'
   -   'Category' -> 'category'
   -   'Size' -> 'size'
   -   'ASIN' -> 'asin'
   -   'Courier Status' -> 'courier_status'
   -   'Qty' -> 'qty'
   -   'Amount' -> 'amount'
   -   'ship-city' -> 'ship_city'
   -   'ship-state' -> 'ship_state'
   -   'ship-postal-code' -> 'ship_postal_code'
   -   'ship-country' -> 'ship_country'
   -   'promotion-ids' -> 'promotion_ids'
   -   'B2B' -> 'b2b'
   -   'fulfilled-by' -> 'fulfilled_by'
   -   'Unnamed: 22' -> 'unnamed:_22'
[INFO] Normalizadas columnas en DF 1:
   -   'DATE' -> 'date'
   -   'Months' -> 'months'
   -   'CUSTOMER' -> 'customer'
   -   'Style' -> 'style'
   -   'SKU' -> 'sku'
   -   'Size' -> 'size'
   -   'PCS' -> 'pcs'
   -   'R

In [ ]:
df_1 = df_list_norm[0].copy()    
df_1

,index,order_id,date,status,fulfilment,sales_channel,ship_service_level,style,sku,category,...,amount,ship_city,ship_state,ship_postal_code,ship_country,promotion_ids,b2b,fulfilled_by,unnamed:_22,file_name
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,647.62,MUMBAI,MAHARASHTRA,"400,081.00",IN,NaN,False,Easy Ship,NaN,Amazon Sale Report.csv
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,406.00,BENGALURU,KARNATAKA,"560,085.00",IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN,Amazon Sale Report.csv
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,329.00,NAVI MUMBAI,MAHARASHTRA,"410,210.00",IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN,Amazon Sale Report.csv
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,...,753.33,PUDUCHERRY,PUDUCHERRY,"605,008.00",IN,NaN,False,Easy Ship,NaN,Amazon Sale Report.csv
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,...,574.00,CHENNAI,TAMIL NADU,"600,073.00",IN,NaN,False,NaN,NaN,Amazon Sale Report.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128970,128970,406-6001380-7673107,05-31-22,Shipped,Amazon,Amazon.in,Expedited,JNE3697,JNE3697-KR-XL,kurta,...,517.00,HYDERABAD,TELANGANA,"500,013.00",IN,NaN,False,NaN,False,Amazon Sale Report.csv
128971,128971,402-9551604-7544318,05-31-22,Shipped,Amazon,Amazon.in,Expedited,SET401,SET401-KR-NP-M,Set,...,999.00,GURUGRAM,HARYANA,"122,004.00",IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN,False,Amazon Sale Report.csv
128972,128972,407-9547469-3152358,05-31-22,Shipped,Amazon,Amazon.in,Expedited,J0157,J0157-DR-XXL,Western Dress,...,690.00,HYDERABAD,TELANGANA,"500,049.00",IN,NaN,False,NaN,False,Amazon Sale Report.csv
128973,128973,402-6184140-0545956,05-31-22,Shipped,Amazon,Amazon.in,Expedited,J0012,J0012-SKD-XS,Set,...,"1,199.00",Halol,Gujarat,"389,350.00",IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN,False,Amazon Sale Report.csv


In [ ]:
df_2 = df_list_norm[1].copy()    
df_2

In [ ]:
df_3 = df_list_norm[2].copy()    
df_3